## For Reviewers

Thanks for reviewing! This project is to teach ray tracing for novices. Our overall coding priority right now is to generate a final image of a dining room scene. We would appreciate feedback on how readable our code is, especially the code creating scene objects and implementing reflections. We would also like to know if you think we prpvide enough background and that the prose is clear.

The outline is:
 - **Background on ray tracing** - unfinished
 - **Scene geometry construction** - unfinished
 - **Ray tracing** - unfinished
 - **Reflection modeling** - unfinished
 - **Enhancements** - unfinished

# Ray Tracing

**Team**: Jade Kessinger, Ryan Nguyen, Lillian Vernooy

**Summary**: Users will be able to perform vector operations on rays that are simulating photons and render a realistic image of some simple object, i.e., spheres, cubes, cylinders, etc. This will provide insight into how we as programmers simplify and approximate real-life physics into something computable with finite memory and time, which is a good step into more complex concepts in computer graphics, such as simulating water, snow, fire, etc.

**Audience**: Students who are familiar with Python and basic vector geometry, but haven't had much experience with computer graphics. These individuals are interested in simulating light in a virtual environment and learning about how computers render images, but not necessarily a complex graphics engine.

**Libraries used**: `NumPy`, `matplotlib`

**Vocabulary**: rays, vector operations, pixels, images, computer graphics, physics, virtual camera, image

## Part 1: Introduction/Motivation

In this section, we will provide an outline of how the ray tracing algorithm works. We want to make sure that the user has a high-level understanding of the process before we begin writing code, so that they understand the purpose of each part of the system as we implement it.

Tasks:

- Provide a high-level overview of the ray tracing process.

In [3]:
# Import statements
import numpy as np
import matplotlib.pyplot as plt

## Part 2: Scene Geometry Construction

Next, we must construct the geometry that we will render. In order to apply ray tracing to the geometry, every object or surface in the scene must be described by a function, so that the intersection (if it exists) of a casted ray can be calculated with each element in the scene. For an arbitrary geometry, this could quickly become complicated, so it will be desirable to create a small set of functions to describe simple geometric primitives, and then to build our scene geometry out of these.

Tasks:

- Write a set of generic functions to describe the surfaces of some set of geometric objects.
- Create a 3D coordinate system in which the scene will reside.
- Build up the scene by using our set of functions to describe actual objects in the space. 

In [5]:
# Sphere geometry computations
def normalize(vector):
    return vector / np.linalg.norm(vector)

def sphere_intersect(center, radius, ray_origin, ray_direction):
    b = 2 * np.dot(ray_direction, ray_origin - center)
    c = np.linalg.norm(ray_origin - center) ** 2 - radius ** 2
    delta = b ** 2 - 4 * c
    if delta > 0:
        t1 = (-b + np.sqrt(delta)) / 2
        t2 = (-b - np.sqrt(delta)) / 2
        if t1 > 0 and t2 > 0:
            return min(t1, t2)
    return None

def nearest_intersected_object(objects, ray_origin, ray_direction):
    distances = [sphere_intersect(obj['center'], obj['radius'], ray_origin, ray_direction) for obj in objects]
    nearest_object = None
    min_distance = np.inf
    for index, distance in enumerate(distances):
        if distance and distance < min_distance:
            min_distance = distance
            nearest_object = objects[index]
    return nearest_object, min_distance

In [6]:
# Creating image and scene
width = 300
height = 200

max_depth = 3

camera = np.array([0, 0, 1])
ratio = float(width) / height
screen = (-1, 1 / ratio, 1, -1 / ratio) # left, top, right, bottom

light = { 'position': np.array([5, 5, 5]), 'ambient': np.array([1, 1, 1]), 'diffuse': np.array([1, 1, 1]), 'specular': np.array([1, 1, 1]) }

image = np.zeros((height, width, 3))

In [7]:
# Creating objects
objects = [
    { 'center': np.array([-0.2, 0, -1]), 'radius': 0.7 },
    { 'center': np.array([0.1, -0.3, 0]), 'radius': 0.1 },
    { 'center': np.array([-0.3, 0, 0]), 'radius': 0.15 }
]

## Part 3: Ray Tracing

Our third task will be to implement the actual ray tracing operation, in which rays are cast outwards to determine where objects will appear in the image. First, we will decide on a point in space to be the camera position, and define a rectangle to be the focal plane. Then, we create an array to represent the image, and map each pixel to a location on the focal plane. Finally, we can determine which object in the scene each pixel in the camera will display, by casting a ray out from the camera through a given pixel in the focal plane, and then recording which object (and the corresponding point in space) that the ray impacts first.

Tasks:

- Define the camera and focal plane positions.
- Create an array to represent the image, and determine the location of each pixel on the focal plane.
- Cast a ray from the camera position through each pixel in the focal plane, and determine the nearest object that each ray intersects (and the intersection point).

In [8]:
# TODO! Not working code (outputs black image)
for i, y in enumerate(np.linspace(screen[1], screen[3], height)):
    for j, x in enumerate(np.linspace(screen[0], screen[2], width)):
        pixel = np.array([x, y, 0])
        origin = camera
        direction = normalize(pixel - origin)

        # check for intersections
        nearest_object, min_distance = nearest_intersected_object(objects, origin, direction)
        if nearest_object is None:
            continue

        # compute intersection point between ray and nearest object
        intersection = origin + min_distance * direction

        # image[i, j] = ...
    print("%d/%d" % (i + 1, height))

45/200
45/200
45/200
45/200
45/200
45/200
45/200
45/200
45/200
45/200
45/200
46/200
46/200
46/200
46/200
46/200
46/200
46/200
46/200
46/200
46/200
46/200
46/200
46/200
46/200
46/200
46/200
46/200
46/200
46/200
46/200
46/200
46/200
46/200
46/200
47/200
47/200
47/200
47/200
47/200
47/200
47/200
47/200
47/200
47/200
47/200
47/200
47/200
47/200
47/200
47/200
47/200
47/200
47/200
47/200
47/200
47/200
47/200
47/200
47/200
47/200
47/200
47/200
47/200
47/200
47/200
47/200
48/200
48/200
48/200
48/200
48/200
48/200
48/200
48/200
48/200
48/200
48/200
48/200
48/200
48/200
48/200
48/200
48/200
48/200
48/200
48/200
48/200
48/200
48/200
48/200
48/200
48/200
48/200
48/200
48/200
48/200
48/200
48/200
48/200
48/200
48/200
48/200
48/200
48/200
49/200
49/200
49/200
49/200
49/200
49/200
49/200
49/200
49/200
49/200
49/200
49/200
49/200
49/200
49/200
49/200
49/200
49/200
49/200
49/200
49/200
49/200
49/200
49/200
49/200
49/200
49/200
49/200
49/200
49/200
49/200
49/200
49/200
49/200
49/200
49/200
49/200
49/200

In [9]:
# Save Image to output
plt.imsave('image.png', image)

## Part 4: Reflection Modeling

Next, we must determine the actual color to assign to each pixel. The first step in this process is to cast a ray from the point of intersection at the object to the light source, in order to determine if the object is shadowed by any other objects. We can then determine the illumination of the point. In the interest of beginning with the simplest possible system, we can begin with a simple diffuse reflection model in which light from the source is scattered in all directions. In this case, the illumination of a point on an object can be calculated based on the angle between the surface normal at that point and the ray which runs from the point to the light source — the cosine of the angle will give us the illumination of the point. The color of the object can also be incorporated at this point. Once we complete this step, we should be able to create a basic rendering of the scene.

Tasks:

- Cast rays to determine whether a given point is in shadow.
- Calculate illumination based on angle between surface and light source.
- Determine final pixel color by combining degree of illumination and color of object.

In [4]:
# TODO!

## Part 5: Enhancements

Once we have created a basic rendering of the scene, we will add enhancements that will improve the photorealism of the scene. Some possibilities are listed below.

Tasks:

- Implement the Phong reflection model to shade objects more accurately by including specular, diffuse, and ambient components.
- Adding multiple bounces in order to render reflections.
- Implement refraction for transparent/translucent objects.
- Implement textures by varying shading parameters across the surface of an object.
- (If time allows) More complicated geometries. For example, import an STL file, and parse it to determine the set of triangular surfaces in 3D space that it represents.

In [5]:
# TODO!